In [2]:
# do not touch this cell 
%load_ext jupyter_record


The jupyter_record extension is already loaded. To reload it, use:
  %reload_ext jupyter_record


# Set-up

Make sure that you have completed the set-up instuctions in Assignment 0. 

Every time you create a Jupyter cell in this assignment, it should be annotated with '%%git_commit' at the top. Please do not touch this line of code. If you do not see this line, make sure that you have moved the 'custom.js' file in the right location!


# Data Preprocessing

In this section, you should load your raw training dataset, and create your features for the model. You can find the data here: [shorturl.at/owMPS](https://shorturl.at/owMPS)

**IMPORTANT**: Download your data and place it in the "data" folder. You may run into errors if you place it anywhere else. You can use pandas to import your data from the csv files.

The data files are as following:

- training.csv: Seen/training data. Contains raw features and prediction class ('DepDel15').
- testing_X.csv: Unseen/testing data. Contains only raw features, and no prediction class. 
- readme.html: Information about the features. 



In [4]:
%%git_commit

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/training.csv')
print(df.head())

   Unnamed: 0  FlightDate            Airline Origin Dest  Cancelled  Diverted  \
0           0  2018-01-23  Endeavor Air Inc.    ABY  ATL      False     False   
1           1  2018-01-24  Endeavor Air Inc.    ABY  ATL      False     False   
2           2  2018-01-25  Endeavor Air Inc.    ABY  ATL      False     False   
3           3  2018-01-26  Endeavor Air Inc.    ABY  ATL      False     False   
4           4  2018-01-27  Endeavor Air Inc.    ABY  ATL      False     False   

   CRSDepTime  CRSElapsedTime  Distance  ...  DestAirportID  DestAirportSeqID  \
0        1202            62.0     145.0  ...          10397           1039707   
1        1202            62.0     145.0  ...          10397           1039707   
2        1202            62.0     145.0  ...          10397           1039707   
3        1202            62.0     145.0  ...          10397           1039707   
4        1400            60.0     145.0  ...          10397           1039707   

   DestCityMarketID  DestC

# Model Prediction

In this section, you should import relevant modeling/scikit-learn libraries and create a prediction model from your features. You should carefuly evaluate how your model performs based on the data you already have.

**Hint**: We are using a real world dataset with low correlation between the raw features and prediction class. You may have to carefully think about how to hand-engineer features from the raw features to capture correlation using a machine learning model.


**Hint 2**: When the instructors ran this assignment, we achieved an accuracy of 0.74 and AUC score of 0.54 on held out data. We do not expect high accuracy for this task, but your goal is to create a prediction model that is better than the simplest baseline - i.e. a model that predictions the majority class for all samples.


In [7]:
%%git_commit
result = df.select_dtypes(include='number')
print(result.head())
result.corr()

   Unnamed: 0  CRSDepTime  CRSElapsedTime  Distance  Year  Quarter  Month  \
0           0        1202            62.0     145.0  2018        1      1   
1           1        1202            62.0     145.0  2018        1      1   
2           2        1202            62.0     145.0  2018        1      1   
3           3        1202            62.0     145.0  2018        1      1   
4           4        1400            60.0     145.0  2018        1      1   

   DayofMonth  DayOfWeek  DOT_ID_Marketing_Airline  ...  OriginCityMarketID  \
0          23          2                     19790  ...               30146   
1          24          3                     19790  ...               30146   
2          25          4                     19790  ...               30146   
3          26          5                     19790  ...               30146   
4          27          6                     19790  ...               30146   

   OriginStateFips  OriginWac  DestAirportID  DestAirportSeqID

,Unnamed: 0,CRSDepTime,CRSElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Marketing_Airline,...,OriginCityMarketID,OriginStateFips,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestStateFips,DestWac,DepDel15,CRSArrTime
Unnamed: 0,1.000000,0.004192,-0.001137,0.014973,NaN,-0.104442,0.001249,-0.001237,0.035465,0.002248,...,-0.004158,0.001970,0.008696,0.005305,0.005305,-0.004193,0.001890,0.008623,0.025905,-0.009303
CRSDepTime,0.004192,1.000000,-0.018662,-0.011046,NaN,-0.004670,-0.004523,-0.000657,0.007018,0.002468,...,-0.047958,-0.023831,-0.009526,0.020999,0.020999,0.056120,0.019610,0.020047,0.172096,0.684914
CRSElapsedTime,-0.001137,-0.018662,1.000000,0.980952,NaN,-0.029770,-0.027870,0.002980,0.012194,0.169677,...,-0.013727,-0.036323,-0.027955,0.069337,0.069337,-0.009711,-0.080376,0.063637,0.013894,0.023189
Distance,0.014973,-0.011046,0.980952,1.000000,NaN,-0.031768,-0.030212,0.002684,0.014067,0.140285,...,-0.004516,-0.072488,0.037417,0.056915,0.056916,-0.004123,-0.072921,0.037487,0.012558,0.017925
Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quarter,-0.104442,-0.004670,-0.029770,-0.031768,NaN,1.000000,0.971604,0.006831,0.007626,-0.036160,...,-0.008607,0.044896,-0.033249,-0.019298,-0.019298,-0.008630,0.044857,-0.033235,-0.019861,0.005742
Month,0.001249,-0.004523,-0.027870,-0.030212,NaN,0.971604,1.000000,0.007348,0.019599,-0.034247,...,-0.008799,0.044507,-0.033058,-0.018352,-0.018351,-0.008829,0.044462,-0.033046,-0.018525,0.005239
DayofMonth,-0.001237,-0.000657,0.002980,0.002684,NaN,0.006831,0.007348,1.000000,0.007943,-0.000532,...,0.000154,-0.000419,0.001562,0.000396,0.000396,0.000121,-0.000482,0.001570,-0.004707,-0.002132
DayOfWeek,0.035465,0.007018,0.012194,0.014067,NaN,0.007626,0.019599,0.007943,1.000000,0.011758,...,0.002801,-0.002172,-0.002451,0.001544,0.001544,0.002712,-0.001995,-0.002608,-0.004929,0.007382
DOT_ID_Marketing_Airline,0.002248,0.002468,0.169677,0.140285,NaN,-0.036160,-0.034247,-0.000532,0.011758,1.000000,...,0.036279,0.033113,-0.133493,-0.015255,-0.015255,0.036301,0.033101,-0.133495,-0.004545,-0.003393


# Final Evaluation

In this section, you should use your prediction model on held out data. For each sample in 'testing_X.csv', generate a binary prediction on whether the flight will be delayed by 15min or more.

**Important**: Create a csv file named that is a list of your predictions. It should be named './data/[CNET_ID]_final_assignment.csv' file, and contain only one row and columns equal to the sample size. 

**IMPORTANT**: Make sure that you create the new file in the "data" folder. You may run into errors if you place it anywhere else. 

Upload the '.csv' file here: [shorturl.at/aox05](https://shorturl.at/aox05)

This is your submission for the project! A part of your assessment will be on performance on this unseen dataset, and whether you can do better than majority class prediction. 

Note that you do not have true classification labels on this dataset.


In [ ]:
%%git_commit
